In [2]:
include("./lib/Image.jl")
include("./lib/TorchVision.jl")
using .TorchVision

In [4]:
using Random

# 乱数初期化
## Random.seed!([rng=GLOBAL_RNG], seed) -> rng
## Random.seed!([rng=GLOBAL_RNG]) -> rng
### `!`付きの関数は第一引数の値を破壊的に変更する
Random.seed!(1234)

# PyTorchの乱数初期化
torch.manual_seed(1234)

PyObject <torch._C.Generator object at 0x7f4b9e7dcc30>

In [5]:
using PyCall

# 訓練用、予測用の画像変換関数を作成する関数
## () -> ((PyObject, String) -> Array{Float32,3})
make_transformer_for_learning() = begin
    resize = 224
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    transform = Dict(
        "train" => make_transformer(
            transforms.RandomResizedCrop(resize; scale=(0.5, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.Normalize(mean, std)
        ),
        "val" => make_transformer(
            transforms.Resize(resize),
            transforms.CenterCrop(resize),
            transforms.Normalize(mean, std)
        )
    )
    return (image::PyObject; phase::String="train") -> transform[phase](image)
end

image_transform_vgg16 = make_transformer_for_learning()

#3 (generic function with 1 method)

In [6]:
# ハリネズミとヤマアラシの画像へのファイルパスのリスト作成
make_dataset_list(phase::String="train") = begin
    hedgehogs = map(
        path -> "./dataset/$(phase)/hedgehog/$(path)",
        readdir("./dataset/$(phase)/hedgehog/")
    )
    porcupines = map(
        path -> "./dataset/$(phase)/porcupine/$(path)",
        readdir("./dataset/$(phase)/porcupine/")
    )
    vcat(hedgehogs, porcupines)
end

train_list = make_dataset_list("train")

585-element Array{String,1}:
 "./dataset/train/hedgehog/118523311_32345c36a2.jpg"    
 "./dataset/train/hedgehog/1241612498_7ab4277d10.jpg"   
 "./dataset/train/hedgehog/126009980_9004803c9e.jpg"    
 "./dataset/train/hedgehog/1274493397_88388552d8.jpg"   
 "./dataset/train/hedgehog/127772208_f65a074ed5.jpg"    
 "./dataset/train/hedgehog/1295991716_4ad47dae66.jpg"   
 "./dataset/train/hedgehog/1296287640_19d39d5b1e.jpg"   
 "./dataset/train/hedgehog/1322807353_6eec9596b3.jpg"   
 "./dataset/train/hedgehog/150464690_e33dd1938d.jpg"    
 "./dataset/train/hedgehog/159959475_fb41beb469.jpg"    
 "./dataset/train/hedgehog/163878245_fd30b5169b.jpg"    
 "./dataset/train/hedgehog/17404099_32851ad117.jpg"     
 "./dataset/train/hedgehog/176380875_d2ad991223.jpg"    
 ⋮                                                      
 "./dataset/train/porcupine/PA210066.JPG"               
 "./dataset/train/porcupine/porcupine_sc108.jpg"        
 "./dataset/train/porcupine/porcupine_sud_america.jpg"  
 "

In [34]:
# ハリネズミとヤマアラシのデータセット作成
@pydef mutable struct Dataset <: torch.utils.data.Dataset
    __init__(self, phase::String="phase") = begin
        pybuiltin(:super)(Dataset, self).__init__()
        self.phase = phase
        self.file_list = make_dataset_list(phase)
    end
    
    __len__(self) = length(self.file_list)
    
    __getitem__(self, index::Int) = begin
        # index番目の画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = image_transform_vgg16(img; phase=self.phase)
        # 画像のラベル名をパスから抜き出す
        label = img_path[length(self.phase) + 12 : length(self.phase) + 19]
        # ハリネズミ: 0, ヤマアラシ: 1
        label = (label == "hedgehog" ? 0 : 1)
        return img_transformed, label
    end
end

train_dataset = Dataset("train")
val_dataset = Dataset("val")

# 動作確認
index = 1
img_transformed, label = train_dataset.__getitem__(index)

(Float32[-0.0458088 -0.319805 … -0.439678 -0.30268; -0.232493 -0.722689 … -1.82563 -1.68557; -0.741264 -0.932985 … -0.706405 -0.566972]

Float32[-0.217056 -0.097183 … -0.508177 -0.491052; -0.442577 -0.60014 … -1.82563 -1.84314; -0.81098 -0.81098 … -0.723834 -0.741264]

Float32[0.913177 -0.234181 … -0.542427 -0.491052; 0.677871 -0.617647 … -1.82563 -1.80812; 0.531068 -0.758693 … -0.706405 -0.688976]

...

Float32[0.810429 0.604932 … -1.77541 -1.72403; 0.572829 0.432773 … -2.01821 -2.01821; 0.26963 0.47878 … -1.7173 -1.69987]

Float32[0.998801 1.66667 … -1.74116 -1.68979; 0.712885 1.69328 … -2.01821 -2.03571; 0.600784 1.82083 … -1.66501 -1.64758]

Float32[1.87216 2.19753 … -1.70691 -1.67266; 1.7458 2.35854 … -2.01821 -2.03571; 1.76854 2.43085 … -1.66501 -1.64758], 0)

In [29]:
# ミニバッチサイズ
batch_size = 32

# DataLoader作成
train_dataloader = torch.utils.data.DataLoader(
    train_dataset; batch_size=batch_size, shuffle=true
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset; batch_size=batch_size, shuffle=true
)

# 辞書にまとめる
dataloaders = Dict(
    "train" => train_dataloader,
    "val" => val_dataloader
)

Dict{String,PyObject} with 2 entries:
  "val"   => PyObject <torch.utils.data.dataloader.DataLoader object at 0x7f4b5…
  "train" => PyObject <torch.utils.data.dataloader.DataLoader object at 0x7f4b5…

In [30]:
# 学習済みVGG-16モデルをロード
net = models.vgg16(pretrained=true)

# VGG-16の最後の全結合出力層の出力ユニットを2個に付け替える
## 出力は ハリネズミ=0, ヤマアラシ=1 の2種類分類
net.classifier[7] = torch.nn.Linear(in_features=4096, out_features=2)

# 訓練モードに設定
net.train()

┌ Warning: `setindex!(o::PyObject, v, i::Integer)` is deprecated, use `set!(o, i - 1, v)` instead.
│   caller = top-level scope at In[30]:3
└ @ Core In[30]:3


PyObject VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17

In [31]:
# 損失関数の定義
criterion = torch.nn.CrossEntropyLoss()

# 転移学習で学習させるパラメータを params_to_update に格納
params_to_update = []

# 学習させるパラメータ名
update_param_names = ["classifier.6.weight", "classifier.6.bias"]

# 学習させるパラメータ以外は勾配計算させない
for (name, param) in net.named_parameters()
    if in(name, update_param_names)
        param.required_grad = true
        push!(params_to_update, param)
        println(name)
    else
        param.required_grad = false
    end
end

# params_to_updateの中身を確認
println("----------")
println(params_to_update)

classifier.6.weight
classifier.6.bias
----------
Any[PyObject Parameter containing:
tensor([[ 0.0006,  0.0061, -0.0034,  ...,  0.0021,  0.0064, -0.0136],
        [-0.0058, -0.0038,  0.0030,  ..., -0.0017,  0.0088, -0.0056]],
       requires_grad=True), PyObject Parameter containing:
tensor([-0.0119, -0.0029], requires_grad=True)]


In [32]:
# 最適化手法の設定
optimizer = torch.optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)

PyObject SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)

In [35]:
# モデル訓練
train_model(net, dataloaders, criterion, optimizer, num_epochs) = begin
    tqdm = pyimport("tqdm").tqdm
    
    # epoch数分ループ
    for epoch = 1:num_epochs
        println("Epoch $(epoch)/$(num_epochs)")
        println("----------")
        
        # epochごとの学習と検証のループ
        for phase in ["train", "val"]
            if phase == "train"
                net.train() # 訓練モードに
            else
                net.eval() # 検証モードに
            end
            
            epoch_loss = 0.0 # epochの損失和
            epoch_corrects = 0 # epochの正解数
            
            # 未学習時の検証性能を確かめるため、最初の訓練は省略
            if epoch == 1 && phase == "train"
                continue
            end
            
            # データローダーからミニバッチを取り出すループ
            for (inputs, labels) in tqdm(dataloaders[phase])
                # optimizer初期化
                optimizer.zero_grad()
                
                # 順伝搬計算
                torch.set_grad_enabled(phase == "train")
                outputs = net(inputs)
                loss = criterion(outpus, labels) # 損失計算
                (max, preds) = torch.max(outputs, 1) # ラベルを予測
                # 訓練時はバックプロパゲーション
                if phase == "train"
                    loss.backward()
                    optimizer.step()
                end
                # イテレーション結果の計算
                epoch_loss += loss.item() * iputs.size(0)
                epoch_corrects += torch.sum(preds == labels.data)
                torch.set_grad_enabled(false)
            end
            
            # epochごとの損失と正解率を表示
            epoch_loss = epoch_loss / length(dataloaders[phase].dataset)
            epoch_acc = epoch_corrects^2 / length(dataloaders[phase].dataset)
            println("$(phase) Loss: $(epoch_loss), Acc: $(epoch_acc)")
        end
    end
end

# 学習・検証を実行
train_model(net, dataloaders, criterion, optimizer, 2)

Epoch 1/2
----------
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)
(3, 224, 224)






  0%|                                                    | 0/3 [00:00<?, ?it/s]

(3, 224, 224)
(3, 224, 224)
(3, 224, 224)


PyCall.PyError: PyError (ccall(#= /home/user/.julia/packages/PyCall/ttONZ/src/pyiterator.jl:11 =# @pysym(:PyIter_Next), PyPtr, (PyPtr,), o)) <class 'IndexError'>
IndexError('Julia exception: BoundsError: attempt to access 80-element Array{String,1} at index [0]\nStacktrace:\n [1] getindex(::Array{String,1}, ::Int64) at ./array.jl:729\n [2] ##__getitem__#381(::PyObject, ::Int64) at ./In[28]:13\n [3] (::getfield(PyCall, Symbol("#f_kw_closure#55")){getfield(Main, Symbol("###__getitem__#381")),Tuple{PyObject,Int64},Array{Tuple{Symbol,Any},1}})() at /home/user/.julia/packages/PyCall/ttONZ/src/callback.jl:36\n [4] _pyjlwrap_call(::Function, ::Ptr{PyCall.PyObject_struct}, ::Ptr{PyCall.PyObject_struct}) at /home/user/.julia/packages/PyCall/ttONZ/src/callback.jl:37\n [5] pyjlwrap_call(::Ptr{PyCall.PyObject_struct}, ::Ptr{PyCall.PyObject_struct}, ::Ptr{PyCall.PyObject_struct}) at /home/user/.julia/packages/PyCall/ttONZ/src/callback.jl:49\n [6] _start(::PyObject) at /home/user/.julia/packages/PyC